###### https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

###### Cross-valitation: https://scikit-learn.org/stable/modules/cross_validation.html
###### Selecting parameters: https://scikit-learn.org/stable/modules/grid_search.html#grid-search

In [ ]:
%run "./../library-imports.ipynb"
%run "./../methods-implementation.ipynb"
%run "./../dataset-assumptions.ipynb"

In [ ]:
class CathegoryEnum(Enum):
    dimensionality_reduction = "Dimensionality Reduction"
    regression = "Regression"
    clustering = "Clustering"
    classification = "Classification"

class RegressionEnum(Enum):
    sgdregressor = 'SGDRegressor'
    lasso = 'Lasso'
    elasticnet = 'ElasticNet'
    ridgeregression = 'RidgeRegression'
    svrlinear = 'SVRlinear'
    svrrbf =  'SVRrbf'
    ensembleregressors = 'EnsembleRegressors'
    
## criar enum pras outras categorias...

In [ ]:
class MethodSelector:
    def getMethodCathegory(dataset):
        if(dataset.getDatasetSize()<50):
            raise ValueError('Get more data','MethodSelector')
        if(dataset.getPredictionType()==None):
            dimensionality = DimensionalityReductionSelector()
            dimensionality.setDataset(dataset)
            return dimensionality
        
        if(dataset.getPredictionType()=='quantity'):
            regression = RegressionSelector()
            regression.setDataset(dataset)
            return regression
        
        if(dataset.getPredictionType()=='category' and dataset.getTargetColumn()==None):
            if(dataset.getDatasetSize()>=10000 and dataset.known_prediction_cathegories==False):
                raise ValueError('Clustering: Large datasets require known cathegories','MethodSelector')
            clustering = ClusteringSelector()
            clustering.setDataset(dataset)
            return clustering
        
        if(dataset.getPredictionType()=='category' and dataset.getTargetColumn()!=None):
            #if(dataset.getTargetColumn() not in dataset.getDatasetFeatures()):
            #    raise ValueError('Classification: Wrong Y column','MethodSelector')
            
            classification = ClassificationSelector()
            classification.setDataset(dataset)
            return classification

        raise ValueError('Tough luck','MethodSelector')
        
    def getNextMethodToExecute(cathegory):
        name = cathegory.getRecommendedMethod()
        method = cathegory.getMethodtoExecute()
        return method

In [ ]:
class TechniqueSelector(ABC):     ##abstarct class
    dataset = Dataset
    bolWorking = True
    currentMethod = None
    cathegory_name = None
    
    @abstractmethod   ##abstract method
    def getRecommendedMethod(Self):
        pass
    @abstractmethod
    def getCathegoryName(Self):
        pass
    @abstractmethod
    def getMethodtoExecute(Self):
        pass
        
    def setDataset(Self,dataset):
        Self.dataset = dataset
    def getDataset(Self):
        return Self.dataset
    def setWorking(Self,bolWorking):
        Self.bolWorking = bolWorking
    def getWorking(Self):
        return Self.bolWorking
    def setCurrentMethod(Self,currentMethod):
        Self.currentMethod = currentMethod
    def getCurrentMethod(Self):
        return Self.currentMethod
        

In [ ]:
class RegressionSelector(TechniqueSelector):   ##implementing parent abstract class by using child class
    def getCathegoryName(self):
        return CathegoryEnum.regression.value
    def getRecommendedMethod(self):
        if(self.getCurrentMethod()==None and self.dataset.getDatasetSize()>=100000):
            self.setCurrentMethod('SGDRegressor')
            return 'SGDRegressor'
        if(self.getCurrentMethod()==None and self.dataset.getDatasetSize()<100000 and self.dataset.isFewFeatures()==True):
            self.setCurrentMethod('Lasso')
            return 'Lasso'
        if(self.getCurrentMethod()=="Lasso"):
            self.setCurrentMethod('ElasticNet')
            return 'ElasticNet'
        if(self.getCurrentMethod()==None and self.dataset.getDatasetSize()<100000 and self.dataset.isFewFeatures()==False):
            self.setCurrentMethod('RidgeRegression')
            return 'RidgeRegression'
        if(self.getCurrentMethod()=="RidgeRegression"):
            self.setCurrentMethod('SVRlinear')
            return 'SVRlinear'
        if(self.getCurrentMethod()=="SVRlinear"):
            self.setCurrentMethod('SVRrbf')
            return 'SVRrbf'
        if(self.getCurrentMethod()=="SVRrbf"):
            self.setCurrentMethod('EnsembleRegressors')
            return 'EnsembleRegressors'
        
        raise ValueError('Tough luck','RegressionSelector')
        
    def getMethodtoExecute(self):
        regression = Regression()
        regression.loadCurrentMethod(methodName=self.getCurrentMethod(),dataset=self.dataset)
        return regression

In [ ]:
class ClassificationSelector(TechniqueSelector):   ##implementing parent abstract class by using child class
    def getCathegoryName(self):
        return CathegoryEnum.classification.value
    def getRecommendedMethod(self):
        #print(self.dataset.getDatasetSize())
        #print(self.dataset.isFewFeatures())
        if(self.getCurrentMethod()==None and self.dataset.getDatasetSize()>=100000):
            self.setCurrentMethod('SGDClassifier')
            return 'SGDClassifier'
        if(self.getCurrentMethod()=='SGDClassifier' and self.dataset.getDatasetSize()>=100000):
            self.setCurrentMethod('kernelApproximation')
            return 'kernelApproximation'
        if(self.getCurrentMethod()==None and self.dataset.getDatasetSize()<100000):
            self.setCurrentMethod('LinearSVC')
            return 'LinearSVC'
        if(self.getCurrentMethod()=='LinearSVC' and self.dataset.getDataType=='text'):
            self.setCurrentMethod('NaiveBayes')
            return 'NaiveBayes'
        if(self.getCurrentMethod()=='LinearSVC' and self.dataset.getDataType!='text'):
            self.setCurrentMethod('KNeighborsClassifier')
            return 'KNeighborsClassifier'
        if(self.getCurrentMethod()=='KNeighborsClassifier' and self.dataset.getDataType!='text'):
            self.setCurrentMethod('SVC')
            return 'SVC'
        if(self.getCurrentMethod()=='SVC' and self.dataset.getDataType!='text'):
            self.setCurrentMethod('EnsembleClassifiers')
            return 'EnsembleClassifiers'
            
        raise ValueError('Tough luck','ClassificationSelector')
        
    def getMethodtoExecute(self):
        print(" helloo")

In [ ]:
class DimensionalityReductionSelector(TechniqueSelector):   ##implementing parent abstract class by using child class
    def getCathegoryName(self):
        return CathegoryEnum.dimensionality_reduction.value
    def getRecommendedMethod(self):
        if(self.getCurrentMethod()==None):
            self.setCurrentMethod('RandomizedPCA')
            return 'RandomizedPCA'
        if(self.getCurrentMethod()=='RandomizedPCA' and self.dataset.getDatasetSize()<10000):
            self.setCurrentMethod('Isomap')
            return 'Isomap'
        if(self.getCurrentMethod()=='Isomap'):
            self.setCurrentMethod('SpectralEmbedding')
            return 'SpectralEmbedding'
        if(self.getCurrentMethod()=='SpectralEmbedding'):
            self.setCurrentMethod('LLE')
            return 'LLE'
        if(self.getCurrentMethod()=='RandomizedPCA' and self.dataset.getDatasetSize()>=10000):
            self.setCurrentMethod('kernelApproximation')
            return 'kernelApproximation'
        
        raise ValueError('Tough luck','DimensionalityReductionSelector')
        
    def getMethodtoExecute(self):
        print(" helloo")

In [ ]:
class ClusteringSelector(TechniqueSelector):   ##implementing parent abstract class by using child class
    def getCathegoryName(self):
        return CathegoryEnum.clustering.value
    def getRecommendedMethod(self):
        if(self.getCurrentMethod()==None and self.dataset.known_prediction_cathegories==True and 
           self.dataset.getDatasetSize()>=10000):
            self.setCurrentMethod('MiniBatchKMeans')
            return 'MiniBatchKMeans'
        if(self.getCurrentMethod()==None and self.dataset.known_prediction_cathegories==True and 
           self.dataset.getDatasetSize()<10000):
            self.setCurrentMethod('KMeans')
            return 'KMeans'
        if(self.getCurrentMethod()=='KMeans'):
            self.setCurrentMethod('SpectralClustering')
            return 'SpectralClustering'
        if(self.getCurrentMethod()=='SpectralClustering'):
            self.setCurrentMethod('GMM')
            return 'GMM'
        if(self.getCurrentMethod()==None and self.dataset.known_prediction_cathegories==False and 
           self.dataset.getDatasetSize()<10000):
            self.setCurrentMethod('MeanShift')
            return 'MeanShift'
        if(self.getCurrentMethod()=='MeanShift'):
            self.setCurrentMethod('VBGMM')
            return 'VBGMM'
        
        raise ValueError('Tough luck','ClusteringSelector')
    
    def getMethodtoExecute(self):
        print(" helloo")

In [ ]:
print("start")